## Question 1 - Extracting Tesla Stock Data Using yfinance

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [3]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [20]:
!pip install -U nbformat

def make_graph(stock_data, revenue_data, stock):
    # Ensure the Date column is in datetime format
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'])
    
    # Filter by valid date ranges
    stock_data_specific = stock_data[stock_data['Date'] <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data['Date'] <= '2021-04-30']
    
    # Create subplots (2 rows: one for stock, one for revenue)
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        subplot_titles=("Historical Share Price", "Historical Revenue"),
        vertical_spacing=0.2
    )
    
    # Stock price line chart
    fig.add_trace(
        go.Scatter(
            x=stock_data_specific['Date'],
            y=stock_data_specific['Close'].astype(float),
            name="Share Price",
            mode='lines'
        ),
        row=1,
        col=1
    )
    
    # Revenue line chart
    fig.add_trace(
        go.Scatter(
            x=revenue_data_specific['Date'],
            y=revenue_data_specific['Revenue'].astype(float),
            name="Revenue",
            mode='lines'
        ),
        row=2,
        col=1
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (USD Millions)", row=2, col=1)
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text=f"{stock} Stock Price and Revenue",
        showlegend=False,
        xaxis_rangeslider_visible=True
    )
    
    fig.show()

  Obtaining dependency information for nbformat from https://files.pythonhosted.org/packages/a9/82/0340caa499416c78e5d8f5f05947ae4bc3cba53c9f038ab6e9ed964e22f1/nbformat-5.10.4-py3-none-any.whl.metadata
  Obtaining dependency information for fastjsonschema>=2.15 from https://files.pythonhosted.org/packages/cb/a8/20d0723294217e47de6d9e2e40fd4a9d2f7c4b6ef974babd482a59743694/fastjsonschema-2.21.2-py3-none-any.whl.metadata
  Obtaining dependency information for jsonschema>=2.6 from https://files.pythonhosted.org/packages/bf/9c/8c95d856233c1f82500c2450b8c68576b4cf1c871db3afac5c34ff84e6fd/jsonschema-4.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=22.2.0 from https://files.pythonhosted.org/packages/3a/2a/7cc015f5b9f5db42b7d48157e23356022889fc354a2813c15934b7cb5c0e/attrs-25.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for jsonschema-specifications>=2023.03.6 from https://files.pythonhosted.org/packages/41/45/1a4ed80516f02155c51f51e8cedb3c190229


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
tsla=yf.Ticker('TSLA')

tesla_data=tsla.history(period='max')
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2 - Extracting Tesla Revenue Data Using Webscraping

In [6]:
tesla_revenue = pd.DataFrame(columns=['Date','Revenue'])
print(tesla_revenue)

Empty DataFrame
Columns: [Date, Revenue]
Index: []


In [7]:
html_data = requests.get(' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm').text
soup = BeautifulSoup(html_data)

In [8]:
table_val = soup.find_all('table')

for row in table_val[1].find('tbody').find_all('tr'):
    all_td = row.find_all('td')
    date = all_td[0].text
    revenue =  all_td[1].text
    tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({"Date":[date],"Revenue":[revenue]})],ignore_index=True)

for i,j in enumerate(tesla_revenue['Revenue']):
    # print(i,j)
    txt = j.replace('$',"")
    txt1 = txt.replace(',',"")
    # pri?nt(txt1)
    tesla_revenue["Revenue"][i]=txt1

tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## Question 3 - Extracting GameStop Stock Data Using yfinance

In [10]:

# Extract GameStop stock data

gme = yf.Ticker('GME')
gme_data = gme.history(period='max')
gme_data.reset_index(inplace=True)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [11]:

# Scrape GameStop revenue data
html_data_2 =requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html').text

soup = BeautifulSoup(html_data_2,'html.parser')
gme_revenue = pd.DataFrame(columns=['Date','Revenue'])
print(gme_revenue)

table_val1 = soup.find_all('table')

for row in table_val1[1].find('tbody').find_all('tr'):
    all_td = row.find_all('td')
    date = all_td[0].text
    revenue =  all_td[1].text
    gme_revenue = pd.concat([gme_revenue,pd.DataFrame({"Date":[date],"Revenue":[revenue]})],ignore_index=True)

for i,j in enumerate(gme_revenue['Revenue']):
    # print(i,j)
    txt = j.replace('$',"")
    txt1 = txt.replace(',',"")
    # pri?nt(txt1)
    gme_revenue["Revenue"][i]=txt1

gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue.tail()


Empty DataFrame
Columns: [Date, Revenue]
Index: []


,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## Question 5 - Tesla Stock and Revenue Dashboard

In [22]:

# Merge Tesla stock and revenue
make_graph(revenue_data=tesla_revenue,stock_data=tesla_data,stock="TSLA")



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Question 6 - GameStop Stock and Revenue Dashboard

In [24]:

# Merge GameStop stock and revenue
make_graph(revenue_data=gme_revenue, stock_data=gme_data, stock='GameStop')


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed